In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit 
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
buf = np.load("../valid_files.npy", allow_pickle=True).item()
health = buf["health"]
mdd = buf["mdd"]

In [3]:
df = pd.read_csv('ml_prepared_data/feats.csv')
df["name"] = df["name"].apply(lambda x: x + ".npy")
df = df[df["name"].isin(mdd) | df["name"].isin(health)]

In [4]:
group = df["name"].apply(lambda x: x.split("_")[1])
del df["name"]
df.shape
df.drop(["Unnamed: 0"],axis=1, inplace=True)
df.head()
Y = df["target"]
del df["target"]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df, Y))
df.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,-0.670814,1.024602,-0.181079,-0.040005,-0.879441,0.499306,-0.031779,0.064894,-0.558290,0.904671,...,0.625418,0.448747,-0.911142,0.087249,-0.218308,-0.844347,-1.116242,-0.801455,-0.288989,-0.741415
1,-0.570608,-0.275122,-0.065787,-0.157283,-0.309561,-0.363840,-0.517131,-0.295642,-0.481091,-0.210727,...,-0.399665,-0.371861,-0.244408,0.268891,-0.030293,-0.653745,1.160930,-0.609302,-0.375334,-1.283762
2,-0.835726,2.130446,1.747577,0.405929,-0.418565,2.075168,1.769109,0.080829,-0.721514,1.287306,...,0.241651,0.353787,-0.854070,0.281799,0.207193,0.416948,-0.951562,0.370003,0.039924,0.808006
3,-0.426501,0.504484,-0.496843,-0.577602,-0.769242,-0.356138,-0.316917,-0.197299,-0.750456,0.343705,...,-0.264160,0.447184,-0.957260,-0.344873,-0.222793,-0.636941,-0.909915,-0.443568,-0.314045,-0.896458
4,-0.914678,-0.135352,0.696986,-0.057103,-0.941550,-0.041816,0.456923,2.744134,-0.986224,0.876991,...,0.428626,2.517878,-1.025444,-0.227627,-0.401797,0.375782,-1.091671,-0.168606,-0.329951,1.931530


In [6]:
splitter = GroupShuffleSplit(test_size=.3, n_splits=2, random_state = 7)
split = splitter.split(df, Y, groups=group)
train_inds, test_inds = next(split)

train = df.iloc[train_inds]
y_train = Y.iloc[train_inds]
train_group = group.iloc[train_inds]

test = df.iloc[test_inds]
y_test = Y.iloc[test_inds]
test_group = group.iloc[test_inds]

test.shape

(869, 84)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

splitter_cv = GroupShuffleSplit(test_size=.2, n_splits=5, random_state = 42)
split_cv = splitter.split(train, y_train, groups=train_group)



In [42]:
from sklearn.linear_model import LogisticRegression
R = LogisticRegression(max_iter = 5000)

grid = {
    'C':np.linspace(0.001, 0.01, 100)
}

gs = GridSearchCV(R, param_grid=grid, cv=split_cv, scoring="accuracy")
gs.fit(train, y_train)

y_gs = gs.predict(test)
print(accuracy_score(y_test, y_gs))
print(gs.best_estimator_)

0.5408515535097813
LogisticRegression(C=0.009545454545454548, max_iter=5000)


In [8]:
KN_t = KNeighborsClassifier()

grid_kn = {
    'n_neighbors':range(1, 20),
    'weights': ('uniform', 'distance'),
    'p':(1,2)
}

gs_knc = GridSearchCV(KN_t, param_grid=grid_kn, cv=split_cv, scoring="accuracy")
gs_knc.fit(train, y_train)
y_knc_gs = gs_knc.predict(test)
print(accuracy_score(y_test, y_knc_gs))
print(gs_knc.best_estimator_)

/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accept

0.5845799769850403
KNeighborsClassifier(n_neighbors=8, p=1, weights='distance')


/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
